In [1]:
import statsmodels as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Marginal structural models (MSM), frequentist approach
## IPTW based MSM ad modum Andrew Heiss
This is a python conversion of Andrew Heiss' blog post on IPTW based MSM's here: https://www.andrewheiss.com/blog/2020/12/03/ipw-tscs-msm/
Andrew's generated data will be used. The dataset simulates the effects of the 6-hour working day policy (binary treatment) and number of vacation days (cont. treatment) on happiness in several (fake) countries. The underlying DAG is this: https://www.andrewheiss.com/blog/2020/12/03/ipw-tscs-msm/index_files/figure-html/dag-complex-1.png

In [5]:
data = pd.read_csv('https://www.andrewheiss.com/blog/2020/12/03/ipw-tscs-msm/happiness_data.csv')

## Binary treatment MSM/IPTW
### Underlying question
What is the effect of implementing the 6-hour working day on happiness?
### IPTW for binary treatment

$\text{unstabilized binary IPW}_{it} = \prod_{t=1}^{t} \frac{1}{P(X_{it} | \bar{X}_{i,t-1}, Y_{i,t-1}, C_{it}, V_{i})}$, where

 - ${i}$ is the individual country
 - ${t}$ it the timestep, ${X_{it}}$ is an observed treatment assignment for ${i}$ at ${t}$
 - $\bar{X}_{i,t-1}$ are the observed treatment assignments for ${i}$ <b>up until</b> ${t-1}$
 - ${Y}_{i,t-1}$ is the observed outcome ${i}$ at ${t-1}$, $C_{it}$ are the time varying confounders for ${i}$ at ${t}$
 - $V_{i}$ are the time invariant (constant) confounders for ${i}$.


<b>Read as:</b> "inverse probability of treatment given all previous treatment assignments, the outcome of interest at the previous timestep, the current time varying confounders and the constant confounders"

However, these weights need to be stabilized. This is done by a modifying the numerator:

$\text{stabilized binary IPW}_{it} = \prod_{t=1}^{t} \frac{P(X_{it} | \bar{X}_{i,t-1}, V_{i})}{P(X_{it})| \bar{X}_{i,t-1}, Y_{i,t-1}, C_{it}, V_{i})}$

### 